In [2]:

from appyter import magic
magic.init(lambda _ = globals: _())

# GeneSet Library Set Appyter
This appyter is designed to perform basic statistics, analysis, and visualizations on a Gene Matrix Transpose (.GMT) file. This will allow bioinformatics researchers to analyze relationships between many different gene sets from several gene set libraries.
 To create your own GMT file, please see Enrichr. Enrichr, hosted by the Ma'ayan Laboratory at Mt. Sinai Icahn School of Medicine, is a collection of geneset libraries. 

In [3]:
import numpy as np 
import pandas as pd
import itertools 
import bokeh
import pathlib
from bokeh.plotting import figure, show, save, output_file
from maayanlab_bioinformatics.enrichment import crisp
# from Umap_Visualization import UMAP_Visualization
import os


install tqdm for progress bar


In [4]:
%%appyter hide_code


{% do SectionField(name='GMTSubmission', title='1. Submit a GMT file', subtitle = 'Sumbit a GMT (Gene Matrix Transpose file) for analysis.', img = 'bulb.png') %}
{% do SectionField (name = 'UMAP visualization', title = '2. Scatterplot Visualization', subtitle= 'Visualize relative Geneset similarities on an interactive scatterplot', img = 'bulb.png') %}
{% do SectionField(name = 'Pairwise Similarity Table', title = '3. GeneSet Pairwise Similarity Table', subtitle = '##TODO : FINISH Explore How Similar a Pair of Gene Sets are to One Another. Use the search bar to search for Terms that are of interest. Use the sorting arrows to see', img = 'bulb.png') %}
{% do SectionField(name = 'Pairwise Similarity Search', title = '4. GeneSet Pair Search', subtitle = '###TODO', img = 'bulb.png')%}
{% do SectionField(name = 'GMT Descriptive Statistics', title = '5. Descriptive Statistics', subtitle = '#TODO', img = 'bulb.png') %}


In [5]:
%%appyter code_exec

{% set gs = TabField(
    name='gs_type',
    label='Gene Sets',
    default='Upload',
    choices={'Upload': [
            FileField(
                name='gs',
                label='Gene Set Files',
                default=[],
                examples={
                    'example.gmt': url_for('static', filename = 'Geneshot_PainGenes.gmt')
                }
            ),
        ],},
    
section = 'GMTSubmission',)%}

gs = {{gs.value[0]}}


```python

gs = None
```

In [6]:
%%appyter code_exec

def load_sets(*files):
    ''' Load a set of files into pairs of labeled sets
    '''
    sets = {}
    for file in map(pathlib.Path, files):
        path = pathlib.Path(file)
        if file.suffix == '.gmt':
            for line in map(str.strip, file.open('r')):
                print(lines)
                

```python
def load_sets(*files):
    ''' Load a set of files into pairs of labeled sets
    '''
    sets = {}
    for file in map(pathlib.Path, files):
        path = pathlib.Path(file)
        if file.suffix == '.gmt':
            for line in map(str.strip, file.open('r')):
                print(lines)
```

In [21]:
load_sets(*gs)